# Otimização de Preços Utilizando Regressão
---

### 6º HACKDAY DA COMUNIDADE DS - 21.22/10/2023

https://www.kaggle.com/competitions/product-price-predicition-20/leaderboard

### Grupo: May The Data Be With You
- Edilson Santos
- Aroldo Brancalhão
- Leonardo Rose
- Manoel Mendonça
- M. Alessandro Fonseca

## Sumário

- [ 0 - Procedimentos Iniciais](#0)
- [ 1 - Descrição dos Dados](#1)
- [ 2 - Feature Engineering](#2)
- [ 3 - EDA](#3)
- [ 4 - Pré-Processamento](#4)
- [ 5 - Machine Learning](#5)
- [ 6 - Previsão Teste](#6)
- [ 7 - Performance do Negócio](#7.)
- [ 8 - Próximos Passos](#8.)

---

# Problema de Negócio

A empresa Dados & Decotes atua em e-commerce no ramo de moda masculina e feminina com roupas, calçados e acessórios. O desafio atual da varejista é otimizar os preços dos seus produtos de modo a maximizar o lucro nas vendas.

O atual sistema de precificação de produtos tem duas etapas. Primeiro é colocada margem fixa de lucro sobre o preço de custo do produto. Na segunda etapa, após 3 meses de avaliação do comportamento das vendas, os preços são ajustados de acordo com as quantidades demandadas de cada produto.

Na terceira etapa de precificação, a ser implantada com base no resultado dos estudos de nossa Equipe de Cientistas de Dados, os preços dos produtos novos e recém lançados no mercado serão otimizado com base nas suas características, tais como marca, categoria, tecido e outras.



## Objetivo

Como analistas e cientistas de dados da Dados & Decotes, o desafio da equipe é encontrar quais deveriam ser estes novos preços, com base neste conjunto histórico de dados contendo os preços e as características dos produtos que alcançaram o maior sucesso nas vendas decorrente dos cálculos de elasticidade de preço.

No presente trabalho, as seguintes ferramentas foram utilizadas:

- Modelo Utilizado - Random Forest
- Métrica - SMAPE - 9.11%

<a id='0'></a>
# 0. PROCEDIMENTOS INICIAIS

## 0.1 Importação de Bibliotecas

In [ ]:
import ast
import optuna
import inflection
import warnings

import pandas                                             as pd
import numpy                                              as np
import matplotlib.pyplot                                  as plt
import seaborn                                            as sns
import sklearn.cluster                                    as ct
import sklearn.metrics                                    as mt

from sklearn.preprocessing                                import TargetEncoder
from sklearn.model_selection                              import train_test_split

from sklearn.metrics                                      import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error 
from sklearn.ensemble                                     import RandomForestRegressor, VotingRegressor
from sklearn.linear_model                                 import LinearRegression
from xgboost                                              import XGBRegressor
from catboost                                             import CatBoostRegressor
from lightgbm                                             import LGBMRegressor

from scipy.stats                                          import chi2_contingency

plt.style.use('ggplot')
sns.set_style('darkgrid')

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

warnings.filterwarnings('ignore')

## 0.2 Funções Auxiliares

### Coletânea de Funções

In [ ]:
# Converte todas as KEYs para maiúsculas.
def newDict( inDict ):
    lista = list()
    for item in inDict:
        for key in item:
            item2 = {}
            item2[ key.upper() ] = item[key]
            lista.append( item2 )
    return lista

#Função Customizada da Métrica SMAPE
def smape(y_pred, y_test):

    smape = 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))
    return smape

#Função para computar todas as metricas
def compute_metrics(model_name, y_pred, y_test):

    y_pred = np.expm1(y_pred)
    y_test = np.expm1(y_test)
                     

    smape = 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))
    mae =mean_absolute_error(y_pred, y_test)
    mape = mean_absolute_percentage_error(y_pred, y_test)
    rmse = mean_squared_error(y_pred, y_test, squared=False)

    df = pd.DataFrame({
        'Model':model_name,
        'MAE':mae,
        'MAPE':mape,
        'RMSE':rmse,
        'SMAPE':smape
    }, index=[0])

    return df

#Função para montar a submissão
def make_submission(X_test_2, y_pred_test):

    path = 'http://menezes.mendonca.nom.br/datasets/hackday6_cds/test.json'
    test_raw = pd.read_json(path, orient='split')
    X_test_submission = X_test_2.copy()
    X_test_submission['pid'] = test_raw['pid']
    X_test_submission['actual_price'] = y_pred_test
    df_submission = X_test_submission[['pid','actual_price']]

    return df_submission

#Função para analisar Correlação de features categóricas
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

### Função para montar dados de TESTE

In [ ]:
#Função para aplicar todas as limpezas nos dados de TESTE
def clean_test( X_test_2, onehot, te ):

    # 2.1. Feature Eng: contar número de produtos
    X_test_2['product_details_count'] = X_test_2['product_details'].apply(lambda x: len(ast.literal_eval(x)) )
    # 2.1. Feature Eng: detalhes -> colunas
    X_test_2['product_details'] = X_test_2['product_details'].str.lower()
    X_test_2['product_details'] = X_test_2['product_details'].apply(lambda x: ast.literal_eval(x))
    X_test_2['product_details'] = X_test_2['product_details'].apply(lambda x: newDict(x))

    # 2.1. Feature Eng: busca todas as chaves
    all_keys = set()
    for data in X_test_2['product_details']:
        for item in data:
            if len( str(item.keys()) ) < 17:  # evita KEY='' ou =' '
                continue
            all_keys.update(item.keys())
    
    for key in all_keys:
        X_test_2[key] = X_test_2['product_details'].apply(lambda x: next((item[key] for item in x if key in item), np.nan))
    
    X_test_2.drop('product_details', axis=1, inplace=True)

    # 2.1. Feature Eng: selecionar as melhores colunas
    details_to_maintain_test = ['pid', '_id',
                       'average_rating',
                       'number_of_reviews',
                       'brand',
                       'category',
                       'crawled_at',
                       'description',
                       'images',
                       'out_of_stock',
                       'avg_delivery_time_days',
                       'seller',
                       'sub_category',
                       'fabrication_time',
                       'title',
                       'product_details_count',
                        "BRAND COLOR",
                        "BRAND FIT",
                        "CLOSURE",
                        "COUNTRY OF ORIGIN",
                        "COLOR",
                        "FABRIC",
                        "FABRIC CARE",
                        "FIT",
                        "GENERIC NAME",
                        "HOODED",
                        "IDEAL FOR",
                        "MODEL NAME",
                        "NECK",
                        "NECK TYPE",
                        "NUMBER OF CONTENTS IN SALES PACKAGE",
                        "OCCASION",
                        "OTHER DETAILS",
                        "PACK OF",
                        "PATTERN",
                        "POCKETS",
                        "REVERSIBLE",
                        "SALES PACKAGE",
                        "SECONDARY COLOR",
                        "SLEEVE",
                        "SLEEVE TYPE",
                        "SIZE",
                        "STYLE CODE",
                        "SUITABLE FOR",
                        "TYPE"
                        ]
    X_test_2 = X_test_2[details_to_maintain_test]

    # 2.2. Feature Eng: criar coluna "número de imagens"
    X_test_2['images'] = X_test_2['images'].apply(lambda x: ast.literal_eval(x))
    X_test_2['number_images'] = X_test_2['images'].apply(lambda x: len(x))

    # 2.3. Feature Eng: Coluna Fabric
    X_test_2['has_cotton']    = X_test_2['FABRIC'].str.contains('cott', case=False, na=False)
    X_test_2['has_polyester'] = X_test_2['FABRIC'].str.contains('poly', case=False, na=False)
    X_test_2['has_lycra']     = X_test_2['FABRIC'].str.contains('lycr', case=False, na=False)
    
    # 2.3. Feature Eng: Coluna Brand Fit
    X_test_2['is_regular'] = X_test_2['BRAND FIT'].str.contains('reg', case=False, na=False)
    X_test_2['is_slim']    = X_test_2['BRAND FIT'].str.contains('slim', case=False, na=False)
    X_test_2['is_fit']     = X_test_2['BRAND FIT'].str.contains('fit', case=False, na=False)

    # 2.4. Feature Eng: Preencher NaNs
    X_test_2 = X_test_2.fillna(0)

    # 3.5. Filtrar colunas
    cols_drop = ['_id','crawled_at','description','title','images']
    X_test_2 = X_test_2.drop(cols_drop, axis=1)

    # 3.5. renomear colunas
    X_test_2.columns = X_test_2.columns.map(lambda x: inflection.parameterize(x, separator='_'))

    # 4.1. One-Hot Encoding
    X_test_2 = pd.get_dummies(X_test_2, columns=onehot)
    X_test_2.columns = X_test_2.columns.map(lambda x: inflection.parameterize(x, separator='_'))

    # 4.2. Target Encoding (para as features do tipo string)
    te_cols = ['brand','seller','sub_category',"fabric",
                        "brand_color",
                        "closure",
                        "other_details",
                        "sales_package",
                        "sleeve_type",
                        "model_name",
                        "fabric_care",
                        "hooded",
                        "pockets",
                        "pack_of",
                        "secondary_color",
                        "style_code",
                        "ideal_for",
                        "reversible",
                        "neck",
                        "generic_name",
                        "brand_fit",
                        "pattern",
                        "sleeve",
                        "country_of_origin",
                        "size",
                        "fit",
                        "number_of_contents_in_sales_package",
                        "suitable_for",
                        "occasion",
                        "color",
                        "neck_type",
                        "type"]

    # 4.2. Transf.STR, aplica TE
    X_test_2[te_cols] = X_test_2[te_cols].astype(str)
    X_test_2[te_cols] = te.transform(X_test_2[te_cols])

    X_test_2 = X_test_2.drop(['pid'], axis=1).copy()

    return X_test_2

### Função de Otimização K-Means

In [ ]:
# CLUSTERIZAÇÃO: Otimização de K-Means
# Entrada:
#     X . . . : Dataset a ser "clusterizado"
#     features: lista de features que comandarão a clusterização
#     min_k . : num.mínimo de clusters
#     max_k . : num.máximo de clusters
#
# Saída: o modelo otimizado

def kmeans_fit( X, features, min_k, max_k ):

    X_kmeans = X.loc[ :, features ]

    n_clusters = np.arange( min_k, max_k, 1 )
    best_ss = 0
    best_k = 0
    best_kmeans_model = None
    first_time = True
    ss_list = []

    for c in n_clusters:
        # define model
        kmeans = ct.KMeans( 
            n_clusters=c, 
            init='random', 
            n_init=10, 
            random_state=0
        )

        # FIT & PREDICT
        labels = kmeans.fit_predict( X_kmeans )

        # performance (the bigger, the better)
        ss_avg = mt.silhouette_score( X_kmeans, labels)
        ss_list.append( ss_avg )

        # Compare result
        if first_time or ss_avg > best_ss:
            first_time = False
            best_ss = ss_avg
            best_k = c
            best_kmeans_model = kmeans
            print(">>> BEST - N.Cluster={}  Silhouette={:.6f}".format(best_k, best_ss))

        #print( "n_clusters={}  - The Avg SS: {}".format( c, ss_avg ) )

    print('---- BEST RESULT ---')
    print("Number of Cluster={}  Silhouette={:.6f}".format(best_k, best_ss))

    plt.plot( n_clusters, ss_list, marker='o' )
    plt.xlabel( 'Número de Clusters K' )
    plt.ylabel( 'Silhouette Score Average' )
    print( best_kmeans_model )

    # retorna: modelo + silhouete_score
    return [ best_kmeans_model, best_ss ]

## 0.3 Carregar Dados

In [ ]:
# Dados de Treino
path = 'http://menezes.mendonca.nom.br/datasets/hackday6_cds/train.json'
df_raw = pd.read_json(path, orient='split')


<a id='1'></a>
# 1. DESCRIÇÃO DOS DADOS

In [ ]:
df_raw.sample(3).T

In [ ]:
df_raw.info()

In [ ]:
df_raw.describe()

In [ ]:
df_raw.nunique()

<a id='2'></a>
# 2. FEATURE ENGINEERING

In [ ]:
df1 = df_raw.copy()

## 2.1 Coluna "Product Details"

### Amostra de "product_details"

In [ ]:
df1['product_details'][0]

### Contar Número de Produtos

In [ ]:
#Usar a função ast.literal_eval() para transformar expressões literais em dicionários
df1['product_details_count'] = df1['product_details'].apply(lambda x: len(ast.literal_eval(x)) )

### Transformar detalhes dos produtos em colunas

In [ ]:
# Forçar tudo em minúsculas
df1['product_details'] = df1['product_details'].str.lower()
# Usar a função ast.literal_eval() para transformar expressões literais em dicionários
df1['product_details'] = df1['product_details'].apply(lambda x: ast.literal_eval(x))

In [ ]:
# Converte todas as KEYs para maiúsculas.
df1['product_details'] = df1['product_details'].apply(lambda x: newDict(x))

In [ ]:
df1.isna().sum()

In [ ]:
# Buscar por todas as chaves dos dicionários
all_keys = set()
for data in df1['product_details']:
    for item in data:
        if len( str(item.keys()) ) < 17:  # evita KEY='' ou =' '
            continue
        all_keys.update(item.keys())

# Para cada chave, crie uma coluna no dataframe e...
# verifique, linha por linha, se essa coluna está presente na chave da dicionário...
# caso esteja no dicionário, insira o valor da chave, senão coloque NAN
for key in all_keys:
    df1[key] = df1['product_details'].apply(lambda x: next((item[key] for item in x if key in item), np.nan))
    
# Drope a coluna usada
df1.drop('product_details', axis=1, inplace=True)


In [ ]:
#Renomear colunas duplicadas
#df1.rename(columns={'Pack of': 'Pack Of 2'}, inplace=True)

#df1['Pack Of'].fillna(df1['Pack Of 2'], inplace=True)

#df1.drop('Pack Of 2', axis=1, inplace=True)

In [ ]:
df1.sample(3).T

### Selecionar As Melhores Colunas

In [ ]:
df1.shape

In [ ]:
df1.isna().sum()

In [ ]:
#Utilizar apenas colunas abaixo de 20.000 NaNs
count_nan = pd.DataFrame(df1.isna().sum()).rename(columns={0:'Numero NaN'})
count_nan.loc[count_nan['Numero NaN'] < 20000,:]

In [ ]:
details_to_maintain = ['pid', '_id',
                       'average_rating',
                       'number_of_reviews',
                       'brand',
                       'category',
                       'crawled_at',
                       'description',
                       'images',
                       'out_of_stock',
                       'avg_delivery_time_days',
                       'seller',
                       'sub_category',
                       'fabrication_time',
                       'title',
                       'actual_price',
                       'product_details_count',
                        "BRAND COLOR",
                        "BRAND FIT",
                        "CLOSURE",
                        "COUNTRY OF ORIGIN",
                        "COLOR",
                        "FABRIC",
                        "FABRIC CARE",
                        "FIT",
                        "GENERIC NAME",
                        "HOODED",
                        "IDEAL FOR",
                        "MODEL NAME",
                        "NECK",
                        "NECK TYPE",
                        "NUMBER OF CONTENTS IN SALES PACKAGE",
                        "OCCASION",
                        "OTHER DETAILS",
                        "PACK OF",
                        "PATTERN",
                        "POCKETS",
                        "REVERSIBLE",
                        "SALES PACKAGE",
                        "SECONDARY COLOR",
                        "SLEEVE",
                        "SLEEVE TYPE",
                        "SIZE",
                        "STYLE CODE",
                        "SUITABLE FOR",
                        "TYPE"
                        ]


df1 = df1[details_to_maintain]

In [ ]:
df1.shape

## 2.2 Criar coluna "Número de Imagens"

In [ ]:
#Coluna images
df1['images'] = df1['images'].apply(lambda x: ast.literal_eval(x))
df1['number_images'] = df1['images'].apply(lambda x: len(x))
df1 = df1.drop(['images'],axis=1)

## 2.3 Criar Colunas com base nos Tecidos e no Tamanho

In [ ]:
#Coluna Fabric
df1['has_cotton']    = df1['FABRIC'].str.contains('cott', case=False, na=False)
df1['has_polyester'] = df1['FABRIC'].str.contains('poly', case=False, na=False)
df1['has_lycra']     = df1['FABRIC'].str.contains('lycr', case=False, na=False)

#Coluna Brand Fit
df1['is_regular'] = df1['BRAND FIT'].str.contains('reg', case=False, na=False)
df1['is_slim']    = df1['BRAND FIT'].str.contains('slim', case=False, na=False)
df1['is_fit']     = df1['BRAND FIT'].str.contains('fit', case=False, na=False)

In [ ]:
df1.shape

## 2.4 Preencher NaNs

In [ ]:
df1 = df1.dropna(subset='actual_price')

In [ ]:
df1 = df1.fillna(0)

In [ ]:
df1.isna().sum()

In [ ]:
df1.nunique()

In [ ]:
df1.shape

In [ ]:
df1['CLOSURE'].unique()

In [ ]:
df1['FIT'].unique()

<a id='3'></a>
# 3. EDA - Análise Exploratória

## 3.1 Analise Univariada

In [ ]:
#Separar dataframes em numerico e categorico
numerical_feat = df1.select_dtypes(include=['float','int'])
categorical_feat = df1.select_dtypes(exclude=['float','int','datetime'])

In [ ]:
numerical_feat.hist(bins=50, figsize=(14,9));

In [ ]:
columns_to_plot = numerical_feat.columns

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(30, 15))
fig.subplots_adjust(hspace=0.5)

#Usar o divmod para encontrar os axes 
for i, column in enumerate(columns_to_plot):
    row, col = divmod(i, 3)
    ax = axes[row, col]
    #plotar todos os boxplots
    sns.boxplot(df1, x=column, ax=ax)
    ax.set_title(column)

plt.show()

## 3.2 Analise Bivariada

In [ ]:
plt.figure(figsize=(9,12))

#Category
plt.subplot(2,1,1)
sns.barplot(df1, x='category', y='actual_price')
plt.title('Preço Médio por Categoria')
plt.xlabel("Categoria")
plt.ylabel("Preço")

#Subcategory
plt.subplot(2,1,2)
sns.barplot(df1, x='actual_price', y='sub_category', orient='horizontal')
plt.title('Preço Médio por Subcategoria')
plt.xlabel("Preço")
plt.ylabel("Sub Categoria")


plt.tight_layout()

In [ ]:
plt.figure(figsize=(17,7))

#Sub Category vendas
plt.subplot(1,2,1)
top5_subcat = df1['sub_category'].value_counts().iloc[:5]
top5_subcat = pd.DataFrame(top5_subcat)
sns.barplot(top5_subcat,x='count', y='sub_category', orient='horizontal')
plt.title("Nº Vendas por Sub Categoria")

#Category Vendas
plt.subplot(1,2,2)
top3_cat = df1['category'].value_counts()
top3_cat = pd.DataFrame(top3_cat)
sns.barplot(top3_cat, x='count', y='category', orient='horizontal')
plt.title("Nº Vendas por Categoria")

plt.tight_layout()

In [ ]:
plt.figure(figsize=(15,6))

#top 10 Brands
plt.subplot(1,2,1)
top10_brands = df1['brand'].value_counts().iloc[:10].index
top_df = df1.loc[df1['brand'].isin(top10_brands),:]
sns.barplot(top_df, y='brand', x='actual_price', orient='horizontal')
plt.title("Top 10 Marcas Mais Vendidas")
plt.xlabel("Nº vendas")
plt.ylabel("Vendedor")

#top 10 Sellers
plt.subplot(1,2,2)
top10_seller = df1['seller'].value_counts().iloc[:10].index
top_df_seller = df1.loc[df1['seller'].isin(top10_seller),:]
sns.barplot(top_df_seller, y='seller', x='actual_price', orient='horizontal')
plt.title("Top 10 Vendedores")
plt.xlabel("Nº Vendas")
plt.ylabel("Vendedor")

plt.tight_layout()

In [ ]:
plt.figure(figsize=(15,9))

#Plotor todos os tipos de tecidos e Sizes
plt.subplot(2,3,1)
sns.barplot(df1, y='actual_price', x='has_polyester')

plt.subplot(2,3,2)
sns.barplot(df1, y='actual_price', x='has_cotton')

plt.subplot(2,3,3)
sns.barplot(df1, y='actual_price', x='has_lycra')

plt.subplot(2,3,4)
sns.barplot(df1, y='actual_price', x='is_slim')

plt.subplot(2,3,5)
sns.barplot(df1, y='actual_price', x='is_regular')

plt.subplot(2,3,6)
sns.barplot(df1, y='actual_price', x='is_fit')

In [ ]:
#Origem de fabricação da roupa
sns.barplot(df1, y='COUNTRY OF ORIGIN', x='actual_price', orient='horizontal')
plt.title("Preço por País de Origem")
plt.xlabel("Preço")
plt.ylabel("País de Origem")


## 3.3 Análise Multivariada

In [ ]:
#Correlação Numéricas
plt.figure(figsize=(10,5))
sns.heatmap(numerical_feat.corr(), annot=True, fmt='.2f', cmap='BrBG')
plt.title("Matriz de Correlação Variáveis Numéricas")

In [ ]:
#Selecionar categóricas para usar no função CRAMER'S V
df_cramer = df1.loc[:,['brand',
                       'category',
                       'sub_category',
                       'actual_price', 
                       'TYPE',
                       'seller',
                       'FABRIC CARE', 
                       'STYLE CODE']]

num_vars = len(df_cramer.columns)
correlation_matrix = np.zeros((num_vars, num_vars))

In [ ]:
#Aplicar o Cramer's V para criar matriz de correlação
for i in range(num_vars):
    for j in range(num_vars):
        if i != j:
            corr = cramers_v(df_cramer.iloc[:, i], df_cramer.iloc[:, j])
            correlation_matrix[i, j] = corr

In [ ]:
plt.figure(figsize=(10,5))

correlation_df = pd.DataFrame(correlation_matrix, columns=df_cramer.columns, index=df_cramer.columns)
sns.heatmap(correlation_df.corr(), annot=True, fmt='.2f', cmap='BrBG')
plt.title("Correlação Variáveis Categóricas");

## 3.4 Hipóteses

### Relação entre Média de Avaliação e Preço

H1 - Produtos com um maior número de avaliações têm mais vendas e, portanto, preços mais elevados. (Resultado: FALSO)

In [ ]:
plt.figure(figsize=(15, 9))  

#Number of Reviews
plt.subplot(2, 2, 1) 
sns.scatterplot(data=df1.loc[df1['number_of_reviews'] > 0,:], x='number_of_reviews', y='actual_price')
plt.title('Número de Avaliações e Preço')
plt.xlabel("Número De Avaliações")
plt.ylabel("Preço")

plt.subplot(2, 2, 2)  
sns.heatmap(df1.loc[df1['number_of_reviews'] > 0,['number_of_reviews', 'actual_price']].corr(), annot=True, cmap='BrBG')
plt.title('Correlação')

#--------------------------------------------------------------#

#Average Rating
plt.subplot(2, 2, 3) 
sns.scatterplot(data=df1.loc[df1['average_rating'] > 0,:], x='average_rating', y='actual_price')
plt.title('Média de Avaliações e Preço')
plt.xlabel("Média De Avaliações")
plt.ylabel("Preço")



plt.subplot(2, 2, 4)  
sns.heatmap(df1.loc[df1['average_rating'] > 0,['average_rating', 'actual_price']].corr(), annot=True, cmap='BrBG')
plt.title('Correlação')

plt.tight_layout()

FALSO. Não temos uma correlação entre Avaliações e Preço.

### Relação entre Descrição e Preço

H2 - Produtos com mais Imagens e detalhes podem ter um preço mais elevado ou vender mais. (Resultado: VERDADEIRO)

In [ ]:
plt.figure(figsize=(15, 9))  

#Number Images
plt.subplot(2, 2, 1) 
sns.scatterplot(data=df1, x='number_images', y='actual_price')
plt.title('Número de Imagens e Preço')
plt.xlabel("Número De Imagens")
plt.ylabel("Preço")

plt.subplot(2, 2, 2)  
sns.heatmap(df1[['number_images', 'actual_price']].corr(), annot=True, cmap='BrBG')
plt.title('Correlação')

#-----------------------------------------------------#

#Product Details Count
plt.subplot(2, 2, 3) 
sns.scatterplot(data=df1, x='product_details_count', y='actual_price')
plt.title('Detalhes do Produtos e Preço')
plt.xlabel("Quantidade de Detalhes")
plt.ylabel("Preço")

plt.subplot(2, 2, 4)  
sns.heatmap(df1[['product_details_count', 'actual_price']].corr(), annot=True, cmap='BrBG')
plt.title('Correlação')

plt.tight_layout()

VERDADEIRO. Temos uma Correlação positiva entre número imagens e o preço, indicadando que itens com preços elevados tendem a ter mais fotos e, consequentemente venderem mais.

### Relação entre Estoque e Preço

H3 - Produtos que estão fora de estoque podem ser precificados de forma diferente. (Resultado: VERDADEIRO)

In [ ]:
sns.barplot(df1, x='out_of_stock', y='actual_price')
plt.title("Relação entre Estoque e Preço")
plt.xlabel("Fora de Estoque")
plt.ylabel("Preço")

VERDADEIRO. Vemos um ligeira diferença entre Produtos em estoque e sem. Pode ser uma estrategia da empresa aumentar os preços de produtos fora de estoque até esse estoque ser renovado.

## 3.5. Filtrar Colunas

In [ ]:
cols_drop = ['_id','crawled_at','description','title']
df1 = df1.drop(cols_drop, axis=1)

#aplicar inflection para renomear colunas
df1.columns = df1.columns.map(lambda x: inflection.parameterize(x, separator='_'))

<a id='4'></a>
# 4. PRÉ-PROCESSAMENTO

In [ ]:
X = df1.drop(['pid','actual_price'], axis=1).copy()
y = df1['actual_price'].copy()

In [ ]:
#20% validação e 80% treino
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 4.1 One-Hot Encoding

In [ ]:
#One Hot Encoding
onehot = ['category','out_of_stock']
X_train = pd.get_dummies(X_train, columns=onehot)
X_val = pd.get_dummies(X_val, columns=onehot)
X = pd.get_dummies(X, columns=onehot)

#Renomear as colunas criadas pelo OneHot Encoding
X_train.columns = X_train.columns.map(lambda x: inflection.parameterize(x, separator='_'))
X_val.columns = X_val.columns.map(lambda x: inflection.parameterize(x, separator='_'))
X.columns = X.columns.map(lambda x: inflection.parameterize(x, separator='_'))

In [ ]:
X_train.shape, X_val.shape

## 4.2 Target Encoding

In [ ]:
X_train.columns

In [ ]:
# Target Encoding (para as features do tipo string)
te_cols = ['brand','seller','sub_category',"fabric",
                        "brand_color",
                        "closure",
                        "other_details",
                        "sales_package",
                        "sleeve_type",
                        "model_name",
                        "fabric_care",
                        "hooded",
                        "pockets",
                        "pack_of",
                        "secondary_color",
                        "style_code",
                        "ideal_for",
                        "reversible",
                        "neck",
                        "generic_name",
                        "brand_fit",
                        "pattern",
                        "sleeve",
                        "country_of_origin",
                        "size",
                        "fit",
                        "number_of_contents_in_sales_package",
                        "suitable_for",
                        "occasion",
                        "color",
                        "neck_type",
                        "type"]

#Tranformar as categorias em strings para aplicar Target Encoding
X_train[te_cols] = X_train[te_cols].astype(str)
X_val[te_cols] = X_val[te_cols].astype(str)
X[te_cols] = X[te_cols].astype(str)

#Criar Target Encoding
te = TargetEncoder(target_type='continuous')

#Aplicar fit_transform no X_train
X_train[te_cols] = te.fit_transform(X_train[te_cols], y_train)

#Transform nos outros
X_val[te_cols] = te.transform(X_val[te_cols])
X[te_cols] = te.transform(X[te_cols])

## 4.3 Kmeans

In [ ]:
# Monta primeira clusterização com: ['brand', 'sub_category', 'type']
features = ['brand', 'sub_category', 'type']
ret = kmeans_fit( X, features, 14, 18 )

model_kmeans1 = ret[0]
ss = ret[1]

# Monta a variável KMEANS e inclui em X
X['kmeans'] = model_kmeans1.labels_

#INCLUI EM X_train e X_val
X_kmeans_train = X_train.loc[ :, features]
X_train['kmeans'] = model_kmeans1.predict( X_kmeans_train )

X_kmeans_val = X_val.loc[ :, features]
X_val['kmeans'] = model_kmeans1.predict( X_kmeans_val )

In [ ]:
# Monta segunda clusterização com: ['average_rating', 'brand_fit', 'sleeve']
features = ['average_rating', 'brand_fit', 'sleeve']
ret = kmeans_fit( X, features, 14, 21 )

model_kmeans2 = ret[0]
ss2 = ret[1]

# Monta a variável KMEANS e inclui em X
X['kmeans2'] = model_kmeans2.labels_

#INCLUI EM X_train e X_val
X_kmeans_train = X_train.loc[ :, features]
X_train['kmeans2'] = model_kmeans2.predict( X_kmeans_train )

X_kmeans_val = X_val.loc[ :, features]
X_val['kmeans2'] = model_kmeans2.predict( X_kmeans_val )

In [ ]:
# Monta terceira clusterização com: ['number_of_contents_in_sales_package', 'neck', 'average_rating']
# OBS: possivelmente incluir também 'secondary_color'

features = ['number_of_contents_in_sales_package', 'neck', 'average_rating']
ret = kmeans_fit( X, features, 27, 30 )

model_kmeans3 = ret[0]
ss3 = ret[1]

# Monta a variável KMEANS e inclui em X
X['kmeans3'] = model_kmeans3.labels_

#INCLUI EM X_train e X_val
X_kmeans_train = X_train.loc[ :, features]
X_train['kmeans3'] = model_kmeans3.predict( X_kmeans_train )

X_kmeans_val = X_val.loc[ :, features]
X_val['kmeans3'] = model_kmeans3.predict( X_kmeans_val )

## 4.4 Transformação em Log

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
sns.histplot(df1['actual_price'], kde=True, bins=25)

plt.subplot(1,2,2)
sns.histplot(np.log1p(df1['actual_price']), kde=True, bins=25)

In [ ]:
#normalizar y
y = np.log1p( y )
y_train = np.log1p( y_train )
y_val = np.log1p( y_val )

## 4.5 Feature Importance

In [ ]:
#Juntar X_train e X_val normalizados
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

In [ ]:
# Observar se alguma feature permanece com string

X_full.sample(5).T

In [ ]:
#<odelo Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=1) 

#Treinamento
rf.fit(X_full, y_full)

#Obtenha a importância das características
feature_importance = rf.feature_importances_

#DataFrame para facilitar a visualização
feature_importance_df = pd.DataFrame({'Feature': X_full.columns, 'Importance': feature_importance})

#Características com base na importância
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

#Gráfico de barras
plt.figure(figsize=(15, 9))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Importância da Característica')
plt.ylabel('Característica')
plt.title('Importância das Características - Random Forest')
plt.show()

<a id='5'></a>
# 5. MACHINE LEARNING

## 5.0 First Trial

### Regressão Linear

In [ ]:
#Instaciar modelo
lr = LinearRegression()

#FIT PREDICT
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_val)

In [ ]:
lr_metrics = compute_metrics('LinearRegression', y_pred_lr, y_val)
lr_metrics

### XGBoost

In [ ]:
#Instaciar modelo
xgb = XGBRegressor()

#FIT PREDICT
xgb.fit(X_train, y_train)
y_pred_val = xgb.predict(X_val)

In [ ]:
xgb_metrics = compute_metrics('XGBoost', y_pred_val, y_val)
xgb_metrics

### Catboost

In [ ]:
#Instaciar modelo
cat = CatBoostRegressor(verbose=False)

#FIT PREDICT
cat.fit(X_train, y_train)
y_pred_cat = cat.predict(X_val)

In [ ]:
cat_metrics = compute_metrics('CatBoost',y_pred_cat, y_val)
cat_metrics

### LightGBM

In [ ]:
#Instaciar modelo
lgb = LGBMRegressor()

#FIT PREDICT
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_val)

In [ ]:
lgb_metrics = compute_metrics('LightGBM',y_pred_lgb, y_val)
lgb_metrics

### Random Forest

In [ ]:
#Instaciar modelo
rf = RandomForestRegressor(n_jobs=-1, random_state=58)

#FIT PREDICT
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)

In [ ]:
rf_metrics = compute_metrics('RandomForest',y_pred_rf, y_val)
rf_metrics

### Ensemble

In [ ]:
#Instaciar modelo
voting_reg = VotingRegressor(
    estimators=[
        ('random_forest', rf),
        ('catboost', cat),
        ('xgboost', xgb),
        ('lighgbm', lgb),
        
    ]
)

#FIT PREDICT
voting_reg.fit(X_train, y_train)
y_pred_vot = voting_reg.predict(X_val)

#Calcular Metricas
ensenble_metrics = compute_metrics('VotingRegressor', y_pred_vot, y_val)
ensenble_metrics

### Resumo das métricas obtidas

In [ ]:
#Juntar todas as metricas
pd.concat([xgb_metrics, lgb_metrics, cat_metrics, rf_metrics, ensenble_metrics, lr_metrics]).sort_values(by='SMAPE').reset_index(drop=True)

In [ ]:
#Parametros usados na random forest
rf.get_params()

## 5.1 Fine Tuning

In [ ]:
#Definir função objetivo do Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1045, 1055, 1),
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        
    }
    #Random Forest com os parâmetros otimizados
    model = RandomForestRegressor(**params, n_jobs=-1, random_state=58)

    #FIT PREDICT
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    #SMAPE
    smape_error = smape(np.expm1(y_pred), np.expm1(y_val))
    
    return smape_error

In [ ]:
study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=5)

In [ ]:
#Melhores Parametros
optuna_best = study.best_params
optuna_best

In [ ]:
fig = optuna.visualization.plot_slice(study)
fig.update_layout(template='plotly_dark', title='<b>Slice Plot', title_x=0.2)

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(template='plotly_dark', title='<b>Optimization History Plot', title_x=0.5)

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.update_layout(template='plotly_dark', title='<b>Hyperparameter Importances', title_x=0.5)

<a id='6'></a>
# 6. PREVISÃO TESTE

## 6.1 Prepara para submissão

In [ ]:
#Modelo Otimizado
n_est = optuna_best.get('n_estimators')
rf = RandomForestRegressor(n_jobs=-1, random_state=58, n_estimators=n_est )
n_est

In [ ]:
rf.fit(X_full, y_full)

In [ ]:
#Ler teste
path = 'http://menezes.mendonca.nom.br/datasets/hackday6_cds/test.json'
X_test_2 = pd.read_json(path, orient='split')

In [ ]:
xxx
X_test_2.sample(4).T

In [ ]:
#Limpar teste
X_test_2 = clean_test(X_test_2, onehot, te )

In [ ]:
# Inclui KMEANS em X_test_2
X_kmeans1 = X_test_2.loc[ :, ['brand', 'sub_category', 'type'] ]
X_test_2['kmeans'] = model_kmeans1.predict( X_kmeans1 )

# Inclui 2º KMEANS
X_kmeans2 = X_test_2.loc[ :, ['average_rating', 'brand_fit', 'sleeve'] ]
X_test_2['kmeans2'] = model_kmeans2.predict( X_kmeans2 )

# Inclui 3º KMEANS
X_kmeans3 = X_test_2.loc[ :, ['number_of_contents_in_sales_package', 'neck', 'average_rating'] ]
X_test_2['kmeans3'] = model_kmeans3.predict( X_kmeans3 )


In [ ]:
#PREDITC
y_pred_test = rf.predict(X_test_2)
y_pred_test = np.expm1(y_pred_test)

## 6.2 Submission

In [ ]:
df_submission = make_submission(X_test_2, y_pred_test)
df_submission

In [ ]:
df_submission.to_csv('submission_mlmm_kmeans_1s2s.csv', index=False)

<a id='7'></a>
# 7. PERFORMANCE DE NEGÓCIO

---

## Premissas

- Vamos considerar uma loja que vende um único produto: Jaquetas de Couro.
- O custo total de cada jaqueta é de R\\$100,00.
- Após a análise de elasticidade de preço, a loja decide que o preço ideal de venda é de R\\$150,00.

## Sem a Otimização do Modelo

- Suponhamos que a loja tenha estabelecido um preço de venda de R\\$150,00 para suas jaquetas de couro.
- Sob essa precificação, a loja vende 1.000 unidades da jaqueta, totalizando R\\$150.000,00 em receita.
- O lucro resultante é de R\\$50.000,00 (R\\$150.000,00 - R$100.000,00 em custos).

## Com o Modelo

Agora, vamos introduzir a otimização de preços com base em um modelo.

- O modelo analisou a base de dados histórica e concluiu que o preço ideal para as jaquetas deveria ser de R\\$200,00.
- Com uma margem de erro de 10% (baseada no treinamento do modelo), a loja pode confiantemente definir o preço em R\\$200,00.
- Como resultado, as vendas não diminuirão significativamente; suponhamos que a loja venda cerca de 900 jaquetas a esse preço.
- O lucro resultante é de R\\$90.000,00 (R\\$180.000,00 - R$90.000,00 em custos), um aumento substancial(80%) em relação à precificação anterior.
  
### Comparação de Cenários:

Sem a otimização do modelo: Receita de R\\$150.000,00 e lucro de R\\$50.000,00.
Com o modelo de otimização: Receita de R\\$180.000,00 e lucro de R\\$90.000,00.


---

<a id='8'></a>
# 8. PRÓXIMOS PASSOS

- Melhorar features do product details
- Implementar otimização baseada no estoque
- Outras Features Engineering
- Implementar outros modelos(Redes Neurais)